Les règles d'association (*association rules*) d'items co-occurents (ex: $A,C => B$) sont minées d'un *dataset* en deux temps: 
1. On recherche les *itemsets* à $k$ éléments ($k$-*itemset*) "les plus fréquents" (on dit de support minimum). Le nombre de combinaisons à dénombre étant exponentiel ($C^{k}_{n}$, avec $n$ le nombre d'*items* uniques), la version *brute force* de cette recherche est souvent bien trop coûteuse et doit être accélérée à l'aide d'algorithmes comme l'algorithme *a priori*.
2. A partir des $k$-*itemsets* minés à l'étape précédente (a priori en petit nombre comparé à $C^{k}_{n}$), on va rechercher les règles d'associations de *confidence* minimums à l'aide d'un algorithme de logique similaire au précédent. Le passage d'un $k$-*itemset* à une règle d'association se fait simplement par partitionnement de l'*itemset* en deux parties, en un LHS (l'antécédent) et un RHS (le conséquent).

On s'est déjà donné deux métriques d'évaluation d'une *association rule*:
* Le *support* qui correspond à sa prévalence dans le *dataset* et qui approxime la probabilité jointe d'occurence des *items* la composant. Par exemple pour une règle $A => B$: $P(A,B)$. 
* La *confidence* qui est le ratio du support de la règle/de l'itemset sur le support de l'antécédent/du LHS. Elle approxime la probabilité conditionnelle du conséquent sachant l'antécedent. Par exemple pour une règle $A => B$: $P(A,B)/P(A)=P(B|A)$. Une bonne association ne peut en effet se contenter d'être fréquemment observée (support élevé) mais doit aussi être informative.

Etape 1: Minage des $k$-itemsets

L'algorithme *a priori* se fonde sur un principe simple: pour un $k$-*itemset* donné, si cet itemset est "fréquent", alors (a priori), chacun de ses sous-itemsets et aussi "fréquent". En particulier, le support d'un itemset ne peut pas excéder le support de chacun de ses sous-itemsets (anti-monotonicité du support). Ce principe va permettre de grandement réduire l'espace des itemsets à explorer.

L'algorithme *a priori* examine les itemsets possibles par taille croissante. L'algorithme est le suivant: 
* Pour chaque $k$ taille d'itemset, initialisé à $1$:
    * Soit C_k l'ensemble des $k$-itemsets candidats (initialisé à C_1 correspondant à l'ensemble des items uniques):
        * Si C_k est vide, c'est terminé, pour le support minimal donné, les plus grands itemsets minés sont de taille $k-1$.
        * Sinon, pour chaque candidat, en déterminer la fréquence dans la database:
            * Si la fréquence déterminée est inférieure au support minimal donné, le candidat n'est pas retenu.
            * Sinon, le candidat est ajouté à la liste des $k$-itemsets fréquents L_k.
            * Une fois tous les candidats examinés: $k=k+1$ ; $C_{k+1}=L_k$ 

On voit que ce principe nous permet de grandement réduire l'espace des itemsets à dénombrer. Par exemple: si un $1$-itemset n'est pas fréquent, aucun des $k$-itemsets pouvant être formé avec celui-ci ne sera examiné aux étapes ultérieures.

Etape 2: Minage des règles d'association
La seconde étape consiste à miner des $k$-itemsets de support minimal obtenus de l'étape précédente, les association rules de confidence minimale. Une rule consiste simplement en le partionnement d'un item set en un LHS (antecedent) et un RHS (consequent), l'ensemble des partitionnements possibles croissant là encore de façon exponentielle avec la taille des itemsets. On va donc recourir à nouveau à un algorithme type a priori.

Là où à l'étape d'avant, le principe "a priori" reposait sur l'anti-monotonicité du support, on va ici se reposer sur l'anti-monotonicité de la *confidence*. Pour un itemset donné, la confidence des association rules formées est d'autant plus faible que le RHS est contient peu d'items.

Par exemple, pour un itemset $(A,B,C,D)$ donné, on voit bien que (finalement comme conséquence de l'anti-monotonicité du support): 

$$c(A => B,C,D) \leq c(A,B => C,D) \leq c(A,B,C => D)$$

Car: 

$$\frac{P(A,B,C,D)}{P(A)} \leq \frac{P(A,B,C,D)}{P(A,B)} \leq \frac{P(A,B,C,D)}{P(A,B,C)}$$

On va ainsi procéder par un algorithme équivalent à l'étape 1, les règles retenues devant présenter une confidence minimale. On part des partitions où le LHS ne présente qu'un élément, si la règle obtenue ne présente pas le niveau de confidence minimal alors (a priori), des règles à LHS plus complexe ne pourront pas obtenir de niveaux de confidence plus élevés et ne seront pas examinées et ainsi de suite.

Une dernière métrique d'évaluation: le lift

Une fois les règles d'association obtenues, on va se donner une métrique supplémentaire: le *lift*.

Le lift correspond au ratio de la confidence sur le support du RHS. Il approxime le ratio de probabilité suivant (pour une règle $A => B$):

$$lift = \frac{P(B|A)}{P(B)} = \frac{P(A,B)}{P(A).P(B)}$$

Le lift vient corriger la possibilité qu'une confidence élevée puisse venir d'une forte représentation du RHS (support élevé du RHS). 

Le lift est ainsi un bon indice de l'existence d'une liaison (positive ou négative) entre l'antécédent et le conséquent d'une règle particulière. D'après les expressions ci-dessus:
* Un lift plus grand que $1$ montre une liaison positive: la probabilité d'occurence de $B$ sachant $A$ est plus élevée que la probabilité incoditionnelle d'occurence de $B$. La deuxième expression autorise une interprétation proche: la probabilité jointe est significativement différente (plus élevée) que si on fait l'hypothèse qu'$A$ et $B$ sont indépendants. Le lift est ainsi une mesure de l'écart à l'indépendance entre antécédent et conséquent.
* Un lift proche de $1$ n'est pas intéressant: l'antécédent est peu informatif du conséquent, la probabilité jointe n'est pas significativement différence de sa valeur sous hypothèse d'indépendence.
* Un lift inférieur à $1$ montre une liaison négative: la probabilité d'occurence de $B$ sachant $A$ est plus faible que la probabilité incoditionnelle d'occurence de $B$. Suivant les use cases, ces règles peuvent ne pas nous intéresser.

L'algorithme a priori est efficace pour trouver des groupes d'items qui co-occurrent  fréquemment dans un *dataset*. Un exemple d'utilisation peut être d'extraire d'information génétiques la co-occurrence de bio-marqueurs. L'exemple classique reste centré sur de la *retail/shopping data* dans laquelle on veut identifier les groupes d'articles qui sont souvent achetés ensembles. Par hypothèse l'algorithme ne fonctionne pas bien pour les articles se vendant peu. L'hypothèse implicite est donc de travailler sur des *best-sellers* ou tout du moins sur des articles s'écoulant un minimum. L'idée est ensuite d'optimiser l'agencement en magasin (qu'on va d'autant moins le changer pour des articles se vendant peu). Ce *use case* s'intéresse donc le plus souvent des points de vente physiques et est moins pertinent voire s'applique mal aux sites d'e-commerce.

Il va falloir se donner un seuil pour se définir ce que "fréquemment" veut dire.

Donnée le plus souvent binaire: $X_i \in \{0,1\}$


 Confidence tells you if the relationship may be random, because the antecedent occurs very often, lift and leverage can tell you if the relationship is coincidental.
 
 . But usually you want all three to be high.

high support: should apply to a large amount of cases
high confidence: should be correct often
high lift: indicates it is not just a coincidence
Consider e.g. "rain" and "day". Assuming we live in a very unfortunate place at the Equator, where it is raining 50% of the time, and it is day 50% of the time, and these are independent of each other. I.e. in 25% of the time it is raining and it is day.

https://cs.wmich.edu/alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part2.pdf